# Physics-Informed Neural Networks

Physics-Informed Neural Networks (PINNs) are special types of Neural Networks that incorporate physical laws into their optimization process. This is usually realized by adding physical laws in the form of differential equations (in implicit form) to the cost function. This allows for the wanted solution not only to minimize the prediction error, but also to satisfy already known physical laws. PINNs allow for the integration of a priori scientific knowledge into data-driven ML models and result in better generalizations. This kind of cross between ML and science is often dubbed "SciML".

PINNs are mostly unsupervised and mesh-free (i.e. continuous in independent variables). Their convergence properties are not yet well understood, and their computational cost is usually much higher when compared to normal neural networks. They are also poor at scaling to larger domains and complex solutions.

This project is based on a crash course on PINNs with a guided demonstration by Ben Moseley that is available at the following address: https://youtu.be/G_hIppUWcsc?si=P1-v2H3RiHQ07PfD.
Some of the Markdown material (especially LateX-based) is borrowed from the author's Notebook.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

: 

### Harmonic Oscillator (Underdamped version)

In [ ]:
def exact_solution(delta, w0, t):
    "Analytical solution to the underdamped h.o."

    assert delta < w0 #for underdampened problem
    
    w = np.sqrt(w0**2 - delta**2)
    phi = np.arctan(-delta/w)
    A = 1 / (2*np.cos(phi))
    cos = torch.cos(w*t + phi)
    exp = torch.exp(-delta * t)
    u = 2 * A * exp * cos
    return u

In [ ]:
class FCN(nn.Module):
    "Defines a standard, fully connected Neural Network in PyTorch"

    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh

        self.fcs = nn.Sequential(*[
            nn.Linear(N_INPUT, N_HIDDEN),
            activation()
        ])

        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()])
                        for _ in range(N_LAYERS-1)
        ])

        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)

    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

### 1. Finding solution to Underdampened Harmonic Oscillator by Training a PINN to simulate the physical system

In [ ]:
torch.manual_seed(123)

def train_PINN_harmonic_oscillator(STEPS):
    "Trains PINN for the underdamped harmonic oscillator problem using known physical law."
    #Defining Neural 
    pinn = FCN(1, 1, 32, 3)

    #Defining boundary loss
    t_boundary = torch.tensor(0.).view(-1, 1).requires_grad_(True)

    #Defining physics loss
    t_physics = torch.linspace(0, 1, 30).view(-1, 1).requires_grad_(True)

    #Training loop for PINN
    d, w0 = 2, 20
    mu, k = 2*d, w0**2
    lambda1 = 1e-1
    lambda2 = 1e-4

    t_test = torch.linspace(0, 1, 300).view(-1,1)
    #Calculating exact solution
    u_exact = exact_solution(d, w0, t_test)
    optimizer = torch.optim.NAdam(pinn.parameters(), lr=1e-3)

    for i in range(STEPS):
        t_boundary = t_boundary.detach().requires_grad_(True)
        t_physics  = t_physics.detach().requires_grad_(True)

        optimizer.zero_grad()

        ### Compute terms in full loss function using hyperparameters ###

        #Compute boundary losses
        u = pinn(t_boundary) #value for u at t=0

        #Loss for initial condition: u(t=0) = 1
        loss1 = (torch.squeeze(u) -1 )**2

        dudt = torch.autograd.grad(u, t_boundary, torch.ones_like(u), create_graph=True)[0] #IMPORTANT DETAIL FOR a tensor of ones that has to to do with the underlying Jacobian computation

        #Loss for initial condition: du/dt(t=0) = 0
        loss2 = (torch.squeeze(dudt) - 0)**2

        #Compute physics losses
        u = pinn(t_physics)
        dudt = torch.autograd.grad(u, t_physics, torch.ones_like(u), create_graph=True)[0]
        d2udt2 = torch.autograd.grad(dudt, t_physics, torch.ones_like(dudt), create_graph=True)[0]
        loss3 = torch.mean((d2udt2 + mu*dudt + k*u)**2)

        #Backpropagation
        loss = loss1 + lambda1 * loss2 + lambda2 * loss3
        loss.backward()
        optimizer.step()

        ### Plot results during training ###
        if  i % 5000 == 0:
            u = pinn(t_test).detach()
            plt.figure(figsize=(6, 2.5))
            plt.scatter(t_physics.detach()[:,0],
                        torch.zeros_like(t_physics)[:,0], s=20, lw=0, color="tab:green", alpha=0.6)
            plt.scatter(t_boundary.detach()[:,0],
                torch.zeros_like(t_boundary)[:,0], s=20, lw=0, color="tab:red", alpha=0.6)
            plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
            plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green", alpha=0.6)
            plt.legend()
            plt.show()

train_PINN_harmonic_oscillator(STEPS = 15001)

The above example is in reality just a solver of a differential equation (law of underdampened harmonic oscillation) using a Neural Network. Nowhere in these examples are we using actual empirical data/observations.

### 2. Training a PINN to invert for underlying parameters (dampening factor $\mu$)